In [21]:
using Plots, ComplexPhasePortrait, ApproxFun, SingularIntegralEquations, DifferentialEquations
gr();

# M3M6: Methods of Mathematical Physics

$$
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\dx{\D x}
\def\dt{\D t}
\def\C{{\mathbb C}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\qqfor{\qquad\hbox{for}\qquad}
$$

Dr. Sheehan Olver
<br>
s.olver@imperial.ac.uk

Office Hours: 3-4pm Mondays, Huxley 6M40
<br>
Website: https://github.com/dlfivefifty/M3M6LectureNotes

# Lecture 14: Logarithmic singular integrals

1. Logarithmic kernels
    - Solving a logarithmic singular integral equation
    - Evaluating logarithmic singular integrals
2. Application: electrostatic potentials in 2D
    - Potential arising from a point charge and a single plate
    
The motivation behind this lecture is to calculate electrostatic potentials. An example is the Faraday cage: imagine a series of metal plates connected together so that they have the same charge. If configured to surround a region, this configuration will shield the interior from an external charge:

![Faradaycage](Laplacetangentialplot.jpg)

Here, the coloured lines are equipotential lines, and there is a point source at $x = 2$, which corresponds to a forcing of 
$$\log\| (x,y)  - (2,0) \| = \log|z - 2|$$
where  $z = x + \I y$. 

## Logarithmic singular integrals

From the Green's function of the Laplacian, it is natural to consider logarithmic singular integrals, and we focus yet again on $[-1,1]$:

$$v(z) = {1 \over \pi }\int_{-1}^1 f(t) \log | z - t| \dt$$

Note that off $[-1,1]$, $v(z)$ solves Laplace's equation, and is continuous on $[-1,1]$:

In [2]:
t = Fun()
f = sqrt(1-t^2)*exp(t)
v = z -> logkernel(f, z)  # logkernel(f,z) calculates 1/π * \int f(t)*log|t-z| dt

xx = yy = linspace(-2.,2., 100)
V = v.(xx' .+ im*yy)

contour(xx, yy, V)
plot!(domain(t); color=:black, label="contour")

In [3]:
surface(xx, yy, V)

Continuity follows since $\log|x-t|$ is integrable for $-1 \leq x \leq 1$ provided $f(t)$ has weaker than pole singularities. 

For $z \notin (-\infty,1]$ this can also be seen since $v$ is the real part of an analytic function:
$$
    v(z) = \Re \int_{-1}^1 f(t) \log ( z - t) \dt
$$
Note that the integrand avoids the branch cut of $\log z$. To extend this to $z\in (-\infty,-1]$ (or more generally, $z \notin [-1,\infty)$), we can use the alternative expression
$$
    v(z) = \Re \int_{-1}^1 f(t) \log (t-z) \dt
$$
which follows from $\log|z-t| = \log|t-z|$.

In [4]:
z = 2.0 +3.0im
@show v(z)
@show sum(f*log(abs(z-t)))/π
@show sum(f*log(z-t))/π
@show sum(f*log(t-z))/π;

v(z) = 0.7068168642791222
sum(f * log(abs(z - t))) / π = 0.7068168642791223
sum(f * log(z - t)) / π = 0.7068168642791222 + 0.5923597267719309im
sum(f * log(t - z)) / π = 0.7068168642791222 - 1.1831399624402499im




### Solving logarithmic singular integral equations

Oddly enough, it is easier to solve a singular integral equation involving the logarithmic kernel than to actually calculate the logarithmic singular integral, so we begin here. Consider the problem of calculating $u(x)$ such that
$$
{1 \over \pi} \int_{-1}^1 u(t) \log | x-t| \dt = f(x) \qqfor -1 < x < 1
$$

Differentiating with respect to $x$, we have
\begin{align*}
{\D\over \dx} \int_{-1}^1 u(t) \log | x-t| \dt  = {\D\over \dx}\left[ \int_{-1}^x u(t) \log (x-t) \dt +  \int_x^1 u(t) \log (t-x) \dt \right] = {\D\over \dx}\left[ \int_{-1}^x u(t) \log (x-t) \dt +  \int_x^1 u(t) \log (t-x) \dt \right]
\end{align*}

Note that, using Leibnitz rule:
\begin{align*}
{\D\over\dx} \int_a^x K(x, t) \dt &=K(x, x) + \int_a^x K_x(x,t)\dt \\
{\D\over\dx} \int_x^b K(x, t) \dt &=-K(x, x) + \int_x^b K_x(x,t)\dt 
\end{align*}
therefore we have
$$
\begin{align*}
{\D\over\dx} \int_{-1}^{x-\epsilon} u(t) \log (x-t) \dt &=u(x) \log \epsilon + \int_{-1}^x {u(t) \over x -t}\dt \\
{\D\over\dx} \int_{x+\epsilon}^1   u(t) \log (t-x) \dt &=-u(x) \log \epsilon  + \int_x^b {u(t) \over x-t}\dt 
\end{align*}
$$
hence letting $\epsilon \rightarrow 0$ we have
$$
{\D\over \dx} {1 \over \pi} \int_{-1}^1 u(t) \log | x-t| \dt  =  {1 \over \pi} \int_{-1}^1 {u(t) \over x-t} \dt = - \HH u(x)
$$
where $\HH$ is again the Hilbert transform.  Thus our original SIE becomes equivalent to inverting the Hilbert transform:
solve
$$
\HH u(x) = -f(x)
$$
recall, we can express the solution as
$$
    u = {1 \over \sqrt{1-x^2}} \HH[\sqrt{1-\diamond^2} f] + {C\over \sqrt{1-x^2}}
$$

Let's do a numerical example:

In [5]:
x = Fun()
f = exp(x)

C = randn()

u₁ = hilbert(sqrt(1-x^2)*f')/sqrt(1-x^2) 

u = u₁ + C/sqrt(1-x^2)

@show hilbert(u, 0.1) + f(0.1)
@show logkernel(u,0.1) - f(0.1)  # didn't work 😩
@show logkernel(u,0.2) - f(0.2);  # but we are only off by a constant

hilbert(u, 0.1) + f(0.1) = -4.440892098500626e-16
logkernel(u, 0.1) - f(0.1) = -1.6854790033751246
logkernel(u, 0.2) - f(0.2) = -1.6854790033751246


Remember: for inverting the Hilbert transform we had a degree of freedom: every solution plus $C/\sqrt{1-x^2}$ was also a solution. But here, since we differentiated, we use that degree of freedom to ensure that we have arrived at the right solution. 

To choose $C$,  we use the fact that
$$
f(0) = {1 \over \pi} \int_{-1}^1 u(t) \log |t| \dt
$$
If we can calculate the right integral, we can use this to choose $C$:

In [6]:
# choose C so that
# logkernel(u₁, 0) + C*logkernel(1/sqrt(1-x^2), 0) == f(0)
C = (f(0) - logkernel(u₁, 0))/logkernel(1/sqrt(1-x^2), 0)
u = u₁ + C/sqrt(1-x^2)

@show hilbert(u, 0.1) + f(0.1)
@show logkernel(u,0.1) - f(0.1)  # Works!
@show logkernel(u,0.2) - f(0.2);  # And at all x!

hilbert(u, 0.1) + f(0.1) = -4.440892098500626e-16
logkernel(u, 0.1) - f(0.1) = 2.220446049250313e-16
logkernel(u, 0.2) - f(0.2) = 2.220446049250313e-16


*Example* We now do an example which can be solved by hand. Find $u(x)$ so that: 
$$
\int_{-1}^1 u(t) \log | x-t| \dt = 1.
$$
Differentiating, we know that 
$$
\int_{-1}^1 {u(t) \over x-t} \dt = 0
$$
hence $u(x)$ must be of the form ${C \over \sqrt{1-x^2}}$.  Which $C$? Make it work for $x = 0$:
$$
1 = \int_{-1}^1 u(t) \log | 0 - t| \dt = C \int_{-1}^1 {\log | t| \over \sqrt{1-t^2}}  \dt
$$
To evaluate the integral, we can use trigonmetric variables:
$$
\int_0^1 {\log t \over \sqrt{1-t^2}}  \dt = \int_0^{\pi \over 2} {\cos \theta \log \sin \theta \over \sqrt{1-\sin^2 \theta}}   \D\theta = \int_0^{\pi \over 2} \log \sin \theta  \D\theta = - {\pi \log 2\over 2}
$$
(The last identity takes some work: I'll leave it as an excercise.)

Thus we have $ C = -{1 \over \pi \log 2}$

In [7]:
x = Fun()
C = -1/(log(2))
u = C/sqrt(1-x^2)

logkernel(u, 0.2) 

1.0

Physically, this solution gives us the potential field 
$$
\int_{-1}^1 u(t) \log|z-t| \dt
$$
corresponding to holding a metal plate at constant potential:

In [8]:
v = z -> π*logkernel(u, z)

xx = yy = linspace(-2.,2., 100)
V = v.(xx' .+ im*yy)

contour(xx, yy, V)
plot!(domain(x); color=:black)

In [9]:
surface(xx, yy, V)

### Evaluating logarithmic singular integrals

How can we actually evaluate 
$$
{1 \over \pi} \int_{-1}^1 u(t) \log|z-t| \dt
$$
for $z$ in the complex plane?  

#### Method 1: integrate Cauchy transform

Consider
$$
g(z) = {1 \over \pi} \int_{-1}^1 u(t) \log(z-t) \dt
$$
so that $v(z) = \Re g(z)$, and note that
$$
g'(z) = {1 \over \pi} \int_{-1}^1 u(t) {1\over z -t}  \dt = -2 \I \CC u(z)
$$
thus if we can calculate $\int^z g'(z) \D z$, we are in good shape. 


_Example_ If $u(x) = {1 \over \sqrt{1-x^2}}$, then 
$$
g'(z) = {1 \over \sqrt{z-1} \sqrt{z+1}} 
$$
and therefore for some constant of integration $D$ we have
$$
g(z) + D = \pi \int^z {\D z \over \sqrt{z-1} \sqrt{z+1}} = 2 \pi \log\left(\sqrt{z-1} + \sqrt{z+1}\right)
$$
We need the right constant of integration. we can find that via the behaviour as $x \rightarrow \infty$:
$$
g(x) ={1 \over \pi}  \int_{-1}^1 u(t) \log(x-t) \dt = {\log x \over \pi}  \int_{-1}^1 u(t) \dt +  {1 \over \pi} \int_{-1}^1 u(t) \log(1-t/x) \dt = {\log x  \over \pi}\int_{-1}^1 u(t) \dt +  O(x^{-1})
$$
Since we have
$$
\int_{-1}^1 {1 \over \sqrt{1-x^2}} \dx = \pi
$$
and
$$
2 \log(\sqrt{x-1} + \sqrt{x+1}) = 2 \log \sqrt x + 2 \log(\sqrt{1-1/x} + \sqrt{1 + 1/x}) = 
\pi \log x + 2\pi \log(2) + O(1/x)
$$
therefore, 
$$
g(z) = 2  \log\left(\sqrt{z-1} + \sqrt{z+1}\right) - 2  \log 2
$$



In [42]:
u = 1/sqrt(1-t^2)
x = 100.0

logkernel(u, 200.0 + im)

5.29832361680194

In [43]:
g = z -> 2*log(sqrt(z-1) + sqrt(z+1)) - 2*log(2)

g(200.0 + im)

5.29832361680194 + 0.005000020832005192im

In particular, for the problem above we have the explicit solution:

In [44]:
g = z -> C*(2*log(sqrt(z-1) + sqrt(z+1)) - 2*log(2))
v = z -> real(g(z))

v(0.2+0.0im)

1.0000000000000002

#### Method 2: represent as Cauchy transform

The approach of Method 1 is not ideal as it involves indefinite integration in the complex plane. However, we will see in the  the next problem sheet that we can actually express $g(z)$ as a Cauchy transform of $\int_x^1 u(t) \dt$. This will use the fact that $g$ satisfies the following Riemann–Hilbert problem:

1. $g(z) \sim {1 \over \pi} \int_{-1}^1 u(t) \dt \log z $
2. $g_+(x) - g_-(x) = 2 \I \int_x^1 u(t) \dt $ for $-1 < x < 1$
3. $g_+(x) - g_-(x) = 2\I \int_{-1}^1 u(t) \dt $ for $x < -1$

_Example_
For the problem above, we have that $\int_x^1 {1 \over \sqrt{1-t^2}}\dt = {\rm arccos}\, x$, which gives us:

In [52]:
u = 1/sqrt(1-t^2)
g = z -> 2*log(sqrt(z-1) + sqrt(z+1)) - 2*log(2)

@show 2im*acos(0.1)
@show g(0.1+0.0im) - g(0.1-0.0im)


@show 2im*sum(u)
@show g(-1.1+0.0im) - g(-1.1-0.0im)

phaseplot(g, (-3,3), (-3,3))

(2im) * acos(0.1) = 0.0 + 2.9412578112666736im
g(0.1 + 0.0im) - g(0.1 - 0.0im) = 0.0 + 2.9412578112666736im
(2im) * sum(u) = 0.0 + 6.283185307179586im
g(-1.1 + 0.0im) - g(-1.1 - 0.0im) = 0.0 + 6.283185307179586im


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3

## Application: Potential arising from a point charge and a single plate

Now imagine we put a point source at $x = 2$, and a metal plate on $[-1,1]$. We know the potential on the plate must be constant, but we don't know what constant. This is equivalent to the following problem:
\begin{align*}
v_xx + v_yy = 0 &\qqfor \hbox{off $[-1,1]$ and $2$}  \\
v(z) \sim \log |z - 2|  &\qqfor z \rightarrow 2 \\
v(z) \sim \log|z| &\qqfor z \rightarrow \infty \\
v(x) = \kappa &\qqfor -1 < x < 1
\end{align*}
where $\kappa$ is an unknown constant.  We write the solution as
$$
v(z) = {1 \over \pi} \int_{-1}^1 u(t) \log|t-z| \dt + \log|z-2|
$$
for a to-be-determined $u$. On $-1 < x < 1$ this satisfies
$$
 {1 \over \pi} \int_{-1}^1 u(t) \log|t-x| \dt  = \kappa - \log(2-x)
$$

In [13]:
x = Fun()
u = SingularIntegral(0)  \ log(2-x)
μ = 
v = z -> log(abs(2-z)) - real(logkernel(u, z))

(::#15) (generic function with 1 method)

In [14]:
xx = yy = linspace(-4.,4., 100)
V = v.(xx' .+ im*yy)

contour(xx, yy, V)
plot!(domain(x); color=:black)

We will see how this can be solved explicitely in the next problem sheet.